In [1]:
{"id":"fUQYdo0BkjfViMQOJMTB","name":"api_key","api_key":"Thac402xRxCzabzQAbdjmg","encoded":"ZlVRWWRvMEJramZWaU1RT0pNVEI6VGhhYzQwMnhSeEN6YWJ6UUFiZGptZw=="}

{'id': 'YyeTbI0BvhpXKm_2q_Gr',
 'name': 'cluster-test',
 'api_key': 'ybnDkr2JSkeO3H90VFqP3Q',
 'encoded': 'WXllVGJJMEJ2aHBYS21fMnFfR3I6eWJuRGtyMkpTa2VPM0g5MFZGcVAzUQ=='}

In [41]:
import warnings
from elasticsearch import Elasticsearch
#Es necesario importar las depencendias necesarias
from datetime import date
from datetime import datetime
import pandas as pd

# Ignore all warnings
warnings.filterwarnings("ignore")   

#b=IJl=v-w67RsCZccMua

class BackupRestore():

    def __init__(self,es_host,es_user,es_password) -> None:    
        # Elasticsearch configuration
        self.es_host = es_host
        self.es_user = es_user
        self.es_password = es_password
        self.es : None

    def Connection(self):
        self.es = Elasticsearch(hosts=[self.es_host], http_auth=(self.es_user, self.es_password), verify_certs=False)
        

    def create_repository(self,repository_name='my_fs_backup'):
        
    
        # Snapshot repository settings
        repository_type = 'fs'
        repository_settings = {
            "location": "/usr/share/elasticsearch/backups",
            "compress": "true"
        }

        # Create a snapshot repository
        response = self.es.snapshot.create_repository(repository_name, body={
            "type": repository_type,
            "settings": repository_settings
        }, request_timeout=30)

        print(response)

    def create_index(self,name):
        if name in self.es.indices.get('*').keys():
            print("Este indice ya esta en uso, pruebe con otro")
        else:
            print("Nuevo indice creado")
            self.es.indices.create(index=name)        

    def list_indices(self):

        indices = self.es.indices.get('*')
        print("Indices:")
        print(indices.keys())

    def insert_doc(self,index_name,doc):
        
        self.es.index(index_name,document=doc)

    def list_snapshots(self):
    # List snapshots
        repository_name = 'my_fs_backup'
        snapshot_repository = self.es.snapshot.get_repository(repository=repository_name)
        snapshot_repository_name = snapshot_repository[repository_name]['type']

        snapshots = self.es.snapshot.get(repository=repository_name, snapshot='_all')
        print("Snapshots:")
        for snapshot in snapshots['snapshots']:
            print(snapshot['snapshot'])

    def list_repositories(self):
        # List snapshot repositories
        repositories = self.es.snapshot.get_repository()
        print("Snapshot Repositories:")
        for repo_name in repositories:
            print(repo_name)

    def create_snapshot(self,index_name):


        # Snapshot repository name
        repository_name = 'my_fs_backup'

        # Create a snapshot for the index
        #Fecha actual
        now = datetime.now()
        fecha_split=str(now).split('.')[0].replace(':','-').split(' ')
        snapshot_name = 'all-indices'+'-snapshot-'+fecha_split[0]+'-'+fecha_split[1]  # Replace with a suitable name for your snapshot
        try:
            response = self.es.snapshot.create(repository=repository_name, snapshot=snapshot_name, body={
                "indices": index_name,
                "include_global_state": False
            }, request_timeout=300)

            print(response)
        except:
            print('No existe tal índice')

    def restore_snapshot(self,snapshot_name,index_name):

        # Snapshot repository name
        repository_name = 'my_fs_backup'
        #self.create_repository()
        # Close the index before restoring (optional)
        try:
            self.es.indices.close(index=index_name)
        except:
            pass


        # Restore the snapshot
        response = self.es.snapshot.restore(repository=repository_name, snapshot=snapshot_name, body={
            "indices": index_name
        }, request_timeout=300)

        print(response)

        # Reopen the index after restoring (optional)
        try:
            self.es.indices.open(index=index_name)
        except:
            pass

    def Validation(self):
        pass

In [2]:
client=BackupRestore('https://localhost:9201/','elastic','elasticpass123',)

In [3]:
client.Connection()

In [4]:
client.create_repository()

{'acknowledged': True}


In [5]:
client.list_snapshots()

Snapshots:
favorite_mangas_snapshot


In [6]:
client.restore_snapshot('favorite_mangas_snapshot','favorite_mangas')

{'accepted': True}


In [49]:
client2=BackupRestore('https://localhost:9202/','cluster_user','mypass',)

In [50]:
client2.Connection()

In [53]:
client2.restore_snapshot('all-indices-snapshot-2024-02-04-17-03-13','index_from_9203')

{'accepted': True}


In [54]:
result=client.es.search(index='index_from_9203',body={'query':{'match_all':{}}})

NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [favorite_mangas]', favorite_mangas, index_or_alias)

In [36]:
client2.es.indices.create(index='index_from_9203')

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'index_from_9203'}

In [44]:
client2.create_snapshot(['favorite_mangas','index_from_9203'])

{'accepted': True}


In [45]:
client2.es.index(index='index_from_9203',body={'inserted from':'9203'})

{'_index': 'index_from_9203',
 '_id': 'pyEmdo0BnzjTPHjVk7rA',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 2, 'failed': 0},
 '_seq_no': 0,
 '_primary_term': 1}